# Collect Data Points

In [1]:
!pip install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 8.6 MB/s eta 0:00:00 MB/s eta 0:00:01m


Load secrets and init some constants

In [ ]:
from github import Github
from google.colab import userdata
GH_TOKEN = userdata.get('GH_TOKEN')
gh = Github(login_or_token=GH_TOKEN)
TOP_HOW_MANY = 50
SEARCH_QUERIES = ('topic:data-science', 'topic:datascience',
                  'data-science', 'datascience',
                  'data science') #search terms for data science repos

Function to search on [GitHub Search Page](https://github.com/search) using the rest api.

[Here](https://github.com/search?q=topic%3Adata-science&type=repositories) is the example of the results of **topic:data-science** search query.

In [ ]:
def execute_gh_search_query(gh: Github, query: str, limit: int):
  return gh.search_repositories(query, sort='stars', order='desc')[:limit]

Append all search results from all used search queries.

In [ ]:
results = []
for query in SEARCH_QUERIES:
  temp = execute_gh_search_query(gh, query, TOP_HOW_MANY)
  for repo in temp:
    results.append(repo)

Deduplicating, then sorting and then selecting the top results.

In [ ]:
results = list(set(results))
results_sorted = sorted(results, key=lambda repo: repo.stargazers_count, reverse=True)
results_sorted = results_sorted[:TOP_HOW_MANY]

Build shallow csv.


In [ ]:
df = pd.DataFrame([{
    'full_name': repo.full_name,
    'url': repo.html_url,
    'stars': repo.stargazers_count,
}])
df.insert(0, 'rank', range(1, len(df) + 1))
df.head()
df.to_csv('shallow_data.csv', index=False)

# Build full csv.


Some utility functions

In [ ]:
from datetime import datetime, timedelta, timezone
import pandas as pd

def get_contribs_in_last_6_months(repo):
    since = datetime.now() - timedelta(days=180)
    contributors = set()
    count = 0
    try:
        for commit in repo.get_commits(since=since):
            if count >= 1000:
                break
            author = commit.author
            if author:
                contributors.add(author.login)
            count += 1
    except Exception as e:
        print(f"Error getting contributors for {repo.full_name}: {e}")
    return len(contributors)

def get_commit_freq(repo, months=6):
    since = datetime.now() - timedelta(days=months*30)
    try:
        commits = []
        for commit in repo.get_commits(sha=repo.default_branch, since=since):
            commits.append(commit)
        if not commits:
            return 0
        return len(commits) / months
    except Exception as e:
        print(f"Error getting commit frequency for {repo.full_name}: {e}")
        return 0

def get_last_commit_date(repo):
    try:
        default_branch = repo.default_branch
        commits = repo.get_commits(sha=default_branch)
        for commit in commits:
            return commit.commit.author.date  # Return the first (latest) commit date
        return None  # No commits found
    except Exception as e:
        print(f"Error getting last commit date for {repo.full_name}: {e}")
        return None


def get_pr_counts(repo, since_days=30):
    from github.PaginatedList import PaginatedList
    since = datetime.now(timezone.utc) - timedelta(days=since_days)
    open_prs = closed_prs = merged_prs = 0

    def count_prs(state):
        count = merged = 0
        # Use per_page=100 for fewer requests
        prs: PaginatedList = repo.get_pulls(state=state, sort='created', direction='desc')
        for pr in prs:
            if pr.created_at < since:
                break  # Stop if PR is older than 'since'
            count += 1
            if state == 'closed' and pr.merged_at and pr.merged_at >= since:
                merged += 1
        return count, merged

    try:
        open_prs, _ = count_prs('open')
        closed_prs, merged_prs = count_prs('closed')
    except Exception as e:
        print(f"Error getting PR counts for {repo.full_name}: {e}")

    return open_prs, closed_prs, merged_prs

Building the csv with all datapoints. (this takes a while)

In [ ]:
from datetime import datetime, timedelta
import pandas as pd

repo_data = []
total_len = len(results_sorted)
for i, repo in enumerate(results_sorted):
    try:
        print(f'Processing {i + 1}/{total_len}...')
        try:
            closed_issues = repo.get_issues(state='closed').totalCount
        except Exception as e:
            print(f"Error getting closed issues for {repo.full_name}: {e}")
            closed_issues = None
        try:
            number_of_releases = repo.get_releases().totalCount
        except Exception as e:
            print(f"Error getting releases for {repo.full_name}: {e}")
            number_of_releases = None
        try:
            total_contributors = repo.get_contributors().totalCount
        except Exception as e:
            print(f"Error getting contributors for {repo.full_name}: {e}")
            total_contributors = None
        try:
            total_languages = len(repo.get_languages())
        except Exception as e:
            print(f"Error getting languages for {repo.full_name}: {e}")
            total_languages = None

        open_prs, closed_prs, merged_prs = get_pr_counts(repo)

        repo_data.append({
            'full_name': repo.full_name,
            'url': repo.html_url,
            'owner': repo.owner.login,
            'license': repo.license.name if repo.license else None,
            'stars': repo.stargazers_count,
            'forks': repo.forks_count,
            'open_issues': repo.open_issues_count,
            'closed_issues': closed_issues,
            'open_prs': open_prs,
            'closed_prs': closed_prs,
            'merged_prs': merged_prs,
            'date_of_last_commit': get_last_commit_date(repo),
            'number_of_releases': number_of_releases,
            'total_contributors': total_contributors,
            'recent_contributors': get_contribs_in_last_6_months(repo),
            'avg_commits_per_month': get_commit_freq(repo, months=6),
        })
    except Exception as e:
        print(f"Error processing {repo.full_name}: {e}")

df = pd.DataFrame(repo_data)
df.insert(0, 'rank', range(1, len(df) + 1))
df.head()

Processing 1/50...
Processing 2/50...
Processing 3/50...
Processing 4/50...
Processing 5/50...
Processing 6/50...
Processing 7/50...
Processing 8/50...
Processing 9/50...
Processing 10/50...
Processing 11/50...
Processing 12/50...
Processing 13/50...
Processing 14/50...
Processing 15/50...
Processing 16/50...
Processing 17/50...
Processing 18/50...
Processing 19/50...
Processing 20/50...
Processing 21/50...
Processing 22/50...
Processing 23/50...
Processing 24/50...
Processing 25/50...
Processing 26/50...
Processing 27/50...
Processing 28/50...
Processing 29/50...
Processing 30/50...
Processing 31/50...
Processing 32/50...
Processing 33/50...
Processing 34/50...
Processing 35/50...
Processing 36/50...
Processing 37/50...
Processing 38/50...
Processing 39/50...
Processing 40/50...
Processing 41/50...
Processing 42/50...
Processing 43/50...
Processing 44/50...
Processing 45/50...
Processing 46/50...
Processing 47/50...
Processing 48/50...
Processing 49/50...
Processing 50/50...


,rank,full_name,url,owner,license,stars,forks,open_issues,closed_issues,open_prs,closed_prs,merged_prs,date_of_last_commit,number_of_releases,total_contributors,recent_contributors,avg_commits_per_month
0,1,microsoft/ML-For-Beginners,https://github.com/microsoft/ML-For-Beginners,microsoft,MIT License,71904,15498,7,0,3,0,0,2025-02-14 08:01:49+00:00,0,129,4,2.000000
1,2,apache/superset,https://github.com/apache/superset,apache,Apache License 2.0,65784,14863,938,0,75,128,109,2025-04-18 20:31:54+00:00,224,424,149,152.000000
2,3,keras-team/keras,https://github.com/keras-team/keras,keras-team,Apache License 2.0,62888,19575,304,0,14,46,40,2025-04-18 18:49:47+00:00,76,407,89,66.666667
3,4,scikit-learn/scikit-learn,https://github.com/scikit-learn/scikit-learn,scikit-learn,"BSD 3-Clause ""New"" or ""Revised"" License",61782,25765,2161,0,45,102,90,2025-04-18 20:12:58+00:00,45,410,133,91.833333
4,5,pandas-dev/pandas,https://github.com/pandas-dev/pandas,pandas-dev,"BSD 3-Clause ""New"" or ""Revised"" License",45171,18437,3706,0,25,62,39,2025-04-18 16:12:46+00:00,109,413,146,81.000000


In [ ]:
df.to_csv('data.csv', index=False)